# Assigning speeches to their proper orator


In [249]:
import pandas as pd 
import numpy as np
import random
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [3]:
#Define custom speech class
class Speech:
    def __init__(self, text, orator):
        self.text = text
        self.orator = orator

# Importing and preparing data

In [5]:
file_name = './data/all_speeches.csv'

df = pd.read_csv(file_name)

#Set each speech entry as custom class
speeches = []
for index, row in df.iterrows():
    speech = {'text':row['text'], 'orator':row['orator']} 
    speeches.append(Speech(speech['text'], speech['orator']))

speeches[99].orator # Change orator to text to access speech text data

'clinton'

# Splitting training and testing sets

In [213]:
seed = 99
train, test = train_test_split(speeches, test_size=.25, random_state=seed)

In [214]:
train_x = [x.text for x in train]
train_y = [x.orator for x in train]

test_x = [x.text for x in test]
test_y = [x.orator for x in test]

# Vectorize with Bags of Words

In [243]:
# Define Bags of Words Count Vectorizer
vectorizer = CountVectorizer()
#vectorizer = TfidfVectorizer()

# Fit vectorizer to training data, and transformed to provide vectors
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

# Final Variables
# train_x_vectors
# train_y

# Classification

In [244]:
categories = ['bush',
              'clinton',
              'obama',
              'trump']

#### K Nearest Neighbor

In [245]:
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(train_x_vectors, train_y)

# Accuracy readouts
print(model,'ACCURACY: ', model.score(test_x_vectors, test_y))
print(f1_score(test_y, model.predict(test_x_vectors), average=None, labels=categories))

KNeighborsClassifier(n_neighbors=3) ACCURACY:  0.3125
[0.36363636 0.28571429 0.35714286 0.14285714]


In [ ]:
# Using GridSearch to check on parameters 


# Saving the model

In [250]:
with open('./knn_model.pkl', 'wb') as f:
    pickle.dump(clf_knn, f)

In [ ]:
# Loading model
# with open('/knn_model.pkl', 'rb') as f:
#     loaded_clf_knn = pickle.load(f)